In [1]:
import { resolve } from 'node:path';
const wallpapersPath = '/home/kival/Pictures/Wallpapers/';
const aspectRatioMinThreshold = 4 / 3;
const aspectRatioMaxThreshold = 40 / 9;
const minX = 2560;

type ImageResult = { path: string; x: number; y: number; marked?: boolean };

function collectAllImages(dir: string): ImageResult[] {
   const results: ImageResult[] = [];
   for (const file of Deno.readDirSync(dir)) {
      if (file.name === 'Ignore') continue;
      if (file.isDirectory) {
         results.push(...collectAllImages(resolve(dir, file.name)));
      }
      if (file.isFile) {
         const path = resolve(dir, file.name);
         const output = new Deno.Command('identify', {
            args: [path],
         }).outputSync();
         const res = new TextDecoder().decode(output.stdout).trim();
         if (res) {
            const out = res.split('\n');
            // const resolution = out
            //    .find((s) => s.includes('Geometry'))!
            //    .slice(out.find((s) => s.includes('Geometry'))!.indexOf(':') + 1)
            //    .matchAll(/\d+/g)
            //    .toArray()
            //    .map(Number);
            const resolution = out[0]
               .slice(path.length + 1)
               .matchAll(/\d+/g)
               .toArray()
               .map(Number);
            results.push({ path, x: resolution[0], y: resolution[1] });
         }
      }
   }
   return results;
}

function filterImages(res: ImageResult): boolean {
   return (
      res.x / res.y >= aspectRatioMinThreshold &&
      res.x / res.y <= aspectRatioMaxThreshold
   );
}

function copyToWallpapers(path: string) {
   return function (res: ImageResult): ImageResult {
      if (res.x < minX) {
         const scale = Math.ceil(minX / res.x);
         console.log('Upscaling', res.path, 'by', scale);
         new Deno.Command('waifu2x-ncnn-vulkan', {
            args: [
               '-i',
               res.path,
               '-o',
               resolve(
                  path,
                  res.path.includes('hentai') ? 'homework' : 'main',
                  'up_' + res.path.split('/').pop()!
               ),
               '-s',
               scale.toString(),
               '-n',
               '2',
            ],
         }).outputSync();
      } else {
         Deno.copyFileSync(
            res.path,
            resolve(
               path,
               res.path.includes('hentai') ? 'homework' : 'main',
               res.path.split('/').pop()!
            )
         );
      }
      return res;
   };
}


In [2]:
const images = [
   '/home/kival/Pictures/Artwork/not hentai',
   '/home/kival/Pictures/Artwork/Touhou Project',
].map((x) => ({
   source: x,
   collection: collectAllImages(x),
}));
images.forEach((group) =>
   console.log(group.collection.length, 'collected from', `"${group.source}"`)
);

var n = images
   .find((group) => !group.source.includes('hentai')).collection.length;
var h = images
   .find((group) => group.source.includes('hentai')).collection.length;

console.log(`1 in ${n / h}`);

172 collected from "/home/kival/Pictures/Artwork/not hentai"
1744 collected from "/home/kival/Pictures/Artwork/Touhou Project"
1 in 10.13953488372093


In [3]:
images.forEach((group) => {
   group.collection.forEach((item) => (item.marked = filterImages(item)));
   console.log(
      group.collection.filter((item) => item.marked).length,
      'marked to use from',
      `"${group.source}"`
   );
});

var n = images
   .find((group) => !group.source.includes('hentai'))
   .collection.filter((item) => item.marked).length;
var h = images
   .find((group) => group.source.includes('hentai'))
   .collection.filter((item) => item.marked).length;

console.log(`1 in ${n / h}`);


73 marked to use from "/home/kival/Pictures/Artwork/not hentai"
496 marked to use from "/home/kival/Pictures/Artwork/Touhou Project"
1 in 6.794520547945205


In [4]:
Deno.removeSync(wallpapersPath, { recursive: true });
Deno.mkdirSync(wallpapersPath);
Deno.mkdirSync(resolve(wallpapersPath, 'main'));
Deno.mkdirSync(resolve(wallpapersPath, 'homework'));
images
   .flatMap((item) => item.collection)
   .filter((item) => item.marked)
   .forEach(copyToWallpapers(wallpapersPath));
console.log('Saved.');


Upscaling /home/kival/Pictures/Artwork/not hentai/0a0652d0358e18859ee2a073988497b7a70e0f71.png by 2
Upscaling /home/kival/Pictures/Artwork/not hentai/4722554a5be0227b497d55189579a5ac458d4c8c.png by 2
Upscaling /home/kival/Pictures/Artwork/not hentai/7aeebd515245e166c0c1db25b8df0f2c.png by 2
Upscaling /home/kival/Pictures/Artwork/not hentai/ace2dcaa4b3d0ce6b86a1934833027e656913294.png by 2
Upscaling /home/kival/Pictures/Artwork/not hentai/b51efcacb1941566dc88dbc76d8766f4518532e3.png by 2
Upscaling /home/kival/Pictures/Artwork/not hentai/baba.png by 3
Upscaling /home/kival/Pictures/Artwork/not hentai/__inubashiri_momiji_touhou_drawn_by_roke_taikodon__768c7fbaaf5e311d52771bb6874be239.jpg by 2
Upscaling /home/kival/Pictures/Artwork/not hentai/__inubashiri_momiji_touhou_drawn_by_wolflong__0b06bcf30c206d23c0fdfcc7a622bb6e.jpg by 2
Upscaling /home/kival/Pictures/Artwork/not hentai/__izayoi_sakuya_touhou_drawn_by_givuchoko__ae7f83fb8e46d14b2246287d50db4f0f.jpg by 2
Upscaling /home/kival/Pictur

In [5]:
import { document } from 'jsr:@ry/jupyter-helper';
import * as Plot from 'npm:@observablehq/plot';

await Deno.jupyter.display(
   Plot.plot({
      marks: [
         Plot.dot(
            images.flatMap((group) =>
               group.collection.map((item) => ({
                  ...item,
                  source: group.source,
                  fill: group.source.includes('hentai')
                     ? item.marked
                        ? '#fff'
                        : '#000'
                     : item.marked
                     ? '#08f'
                     : '#000',
               }))
            ),
            {
               x: 'x',
               y: 'y',
               fill: 'fill',
               title: 'path',
            }
         ),
      ],
      grid: '#000',
      labelArrow: 'right',
      subtitle: 'Resolution',
      document,
   })
);


Resolution 1,000 2,000 3,000 4,000 5,000 6,000 7,000 8,000 y → 0 1,000 2,000 3,000 4,000 5,000 6,000 7,000 8,000 9,000 10,000 x → /home/kival/Pictures/Artwork/not hentai/__kaenbyou_rin_touhou_drawn_by_fuuzasa__91a7e26d14e85855034ff919353dcf54.png /home/kival/Pictures/Artwork/not hentai/0a0652d0358e18859ee2a073988497b7a70e0f71.png /home/kival/Pictures/Artwork/not hentai/11bc5cbbc9c0ccc72138e15706a8c04d01559848.jpg /home/kival/Pictures/Artwork/not hentai/427b798fc2d87c1ed8ac07081f082809e4fa648a.png /home/kival/Pictures/Artwork/not hentai/4722554a5be0227b497d55189579a5ac458d4c8c.png /home/kival/Pictures/Artwork/not hentai/4eca285148fe99795fa8a2d15fb0a2b750fee875.png /home/kival/Pictures/Artwork/not hentai/7aeebd515245e166c0c1db25b8df0f2c.png /home/kival/Pictures/Artwork/not hentai/ace2dcaa4b3d0ce6b86a1934833027e656913294.png /home/kival/Pictures/Artwork/not hentai/b51efcacb1941566dc88dbc76d8766f4518532e3.png /home/kival/Pictures/Artwork/not hentai/baba.png /home/kival/Pictures/Artwork/not hentai/__inubashiri_momiji_touhou_drawn_by_hihiyama_yokikana__e567aa1c6cd08f2e8b8816e21344ae24.png /home/kival/Pictures/Artwork/not hentai/__inubashiri_momiji_touhou_drawn_by_kaidama__0b8c8e826cd99ed4a6ab6a97965263d1.png /home/kival/Pictures/Artwork/not hentai/__inubashiri_momiji_touhou_drawn_by_roke_taikodon__768c7fbaaf5e311d52771bb6874be239.jpg /home/kival/Pictures/Artwork/not hentai/__inubashiri_momiji_touhou_drawn_by_wolflong__0b06bcf30c206d23c0fdfcc7a622bb6e.jpg /home/kival/Pictures/Artwork/not hentai/__izayoi_sakuya_touhou_drawn_by_givuchoko__ae7f83fb8e46d14b2246287d50db4f0f.jpg /home/kival/Pictures/Artwork/not hentai/__junko_touhou_drawn_by_oshiaki__da6ebeb8c8fda9ff21451cbaa0fdd8b8.png /home/kival/Pictures/Artwork/not hentai/__junko_touhou_drawn_by_ouma_tokiichi__2c1a32f8921f9a82b8059681df22aa87.jpg /home/kival/Pictures/Artwork/not hentai/__kaenbyou_rin_and_reiuji_utsuho_touhou_drawn_by_fuuzasa__4bccd9a5178774c586fef9cd53ac8de1.jpg /home/kival/Pictures/Artwork/not hentai/__kaenbyou_rin_touhou_drawn_by_fuji_norihiro__5e30495b0a8d90116fe80da81de852d4.jpg /home/kival/Pictures/Artwork/not hentai/__kaenbyou_rin_touhou_drawn_by_fuuzasa__445ba476bd5aff63d54ac9e4d4bead8a.jpg /home/kival/Pictures/Artwork/not hentai/eacc2c2fb6a1082c6f79b232e6e04ae9.png /home/kival/Pictures/Artwork/not hentai/__fujiwara_no_mokou_touhou_drawn_by_amamitsu_kousuke__96b1e5992a1a34858948943e69f70515.jpg /home/kival/Pictures/Artwork/not hentai/__fujiwara_no_mokou_touhou_drawn_by_hibaeru__634350affb1a45f8be34c5dbc5748ecc.jpg /home/kival/Pictures/Artwork/not hentai/__hata_no_kokoro_touhou_drawn_by_kaede_mmkeyy__5c62825d4043d87898841ece6cea2b19.png /home/kival/Pictures/Artwork/not hentai/__hinanawi_tenshi_touhou_drawn_by_momoiro_lettuce__6497efe9296fb800ad0971e4def49405.jpg /home/kival/Pictures/Artwork/not hentai/__houjuu_nue_touhou_drawn_by_ootsuki_wataru__6bf0b3be994d591321f9c2e50a8030c0.jpg /home/kival/Pictures/Artwork/not hentai/__inubashiri_momiji_touhou_drawn_by_hammer_sunset_beach__131d1c998d2a6ad1228832f71e7006f4.jpg /home/kival/Pictures/Artwork/not hentai/__fujiwara_no_mokou_touhou_drawn_by_amamitsu_kousuke__baca98a49a41464e97a1c95dbd62d6aa.jpg /home/kival/Pictures/Artwork/not hentai/__fujiwara_no_mokou_touhou_drawn_by_asutora__e1d19220706e8b595d9554585df74e5a.jpg /home/kival/Pictures/Artwork/not hentai/__fujiwara_no_mokou_touhou_drawn_by_envenomist_gengoroumaru__400195eb9cc4b27b7375d736afcfe1a3.jpg /home/kival/Pictures/Artwork/not hentai/__fujiwara_no_mokou_touhou_drawn_by_hammer_sunset_beach__375393bc374e8a24d89d34851b1841da.jpg /home/kival/Pictures/Artwork/not hentai/__fujiwara_no_mokou_touhou_drawn_by_hiiro60__375fa374d5c7c9a18cb3971dfc3a74aa.jpg /home/kival/Pictures/Artwork/not hentai/__fujiwara_no_mokou_touhou_drawn_by_kzdlz__ec5c8b221e317f98195442c77364eb6f.png /home/kival/Pictures/Artwork/not hentai/__fujiwara_no_mokou_touhou_drawn_by_mino_minori__f22edd99c6d47c4e09a13d57bc74903d.png /home/kival/Pictures/Artwork/not hentai/__fujiwara_no_mokou_touhou_drawn_by